# Automatic Seismic to Well Tie 自动化地震到井的标定

#### Import packages 导入包


In [ ]:
import os
import sys
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from wtie import autotie, grid, viz
from wtie.utils.datasets import tutorial

# uncomment if your browser supports it
# %matplotlib notebook

### Import data 导入数据

In this tutorial we work on data from the Well _Torosa1_ of the open [Poseidon](https://terranubis.com/datainfo/NW-Shelf-Australia-Poseidon-3D) dataset. Data is saved in the folder **data/tutorial**.
在本教程中，我们使用来自开放的 Poseidon 数据集的 Torosa1 井的数据。数据保存在 data/tutorial 文件夹中。


In [ ]:
# data path
folder = Path("../data/tutorial")
assert folder.exists()

inputs = tutorial.load_poseidon_data(folder, well="torosa1")

### Load pretrained network to extract the wavelet 加载预训练网络以提取子波


In [ ]:
# neural network's weights
model_state_dict = folder / "trained_net_state_dict.pt"
assert model_state_dict.is_file()

# network training paramters
with open(folder / "network_parameters.yaml", "r") as yaml_file:
    training_parameters = yaml.load(yaml_file, Loader=yaml.Loader)

wavelet_extractor = tutorial.load_wavelet_extractor(training_parameters, model_state_dict)

### Load synthetic modeling tool 加载合成记录建模工具


In [ ]:
modeler = tutorial.get_modeling_tool()

### Parameters for the search 搜索参数


In [ ]:
print(autotie.tie_v1.__doc__)

#### Define the tie parameters search space (optional) 定义标定参数的搜索空间（可选）

So far, supported parameters correspond to the log filtering and depth to time table bulk shift. See the `tie.get_default_search_space_v1` docstring.
目前，支持的参数对应于测井滤波和时深关系表的整体时移。请参阅 tie.get_default_search_space_v1 的文档字符串。


In [ ]:
print(autotie.get_default_search_space_v1.__doc__)

In [ ]:
# Logs processing
median_length_choice = dict(
    name="logs_median_size", type="choice", values=[i for i in range(11, 63, 2)], value_type="int"
)

median_th_choice = dict(name="logs_median_threshold", type="range", bounds=[0.1, 5.5], value_type="float")

std_choice = dict(name="logs_std", type="range", bounds=[0.5, 5.5], value_type="float")


# bulk shift in seconds
table_t_shift_choice = dict(name="table_t_shift", type="range", bounds=[-0.012, 0.012], value_type="float")


search_space = [median_length_choice, median_th_choice, std_choice, table_t_shift_choice]

#### Bayesian search parameters (optional) 贝叶斯搜索参数（可选）


In [ ]:
search_params = dict(num_iters=80, similarity_std=0.02)

#### Wavelet absolute amplitude search params 子波绝对振幅搜索参数


In [ ]:
wavelet_scaling_params = dict(wavelet_min_scale=50000, wavelet_max_scale=500000, num_iters=60)

### Optimization 优化


In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")


outputs = autotie.tie_v1(
    inputs,
    wavelet_extractor,
    modeler,
    wavelet_scaling_params,
    search_params=search_params,
    search_space=search_space,  # type: ignore
    stretch_and_squeeze_params=None,  # type: ignore
)

#### QC optimization 质量控制优化


In [ ]:
best_parameters, values = outputs.ax_client.get_best_parameters()  # type: ignore
means, covariances = values  # type: ignore
print(means)
print(covariances)

In [ ]:
pprint(best_parameters)

In [ ]:
outputs.plot_optimization_objective();

### Visualize results


In [ ]:
fig, axes = outputs.plot_wavelet(fmax=85, phi_max=15, figsize=(6, 5))
axes[0].set_xlim((-0.1, 0.1))
axes[2].set_ylim((-12.5, 12.5))
fig.tight_layout()

In [ ]:
_scale = 120000
fig, axes = outputs.plot_tie_window(wiggle_scale=_scale, figsize=(12.0, 7.5))

In [ ]:
fig, ax = viz.plot_td_table(inputs.table, plot_params=dict(label="original"))
viz.plot_td_table(outputs.table, plot_params=dict(label="modified"), fig_axes=(fig, ax))
ax.legend(loc="best")